In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF,WhiteKernel,DotProduct
import pandas as pd
import torch

In [2]:
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass
from snorkel.annotations import load_gold_labels
from snorkel.learning.pytorch import LSTM
from scipy import vstack

In [3]:
lstm_h = LSTM(n_threads=None)
lstm_l = LSTM(n_threads=None)

In [4]:
lstm_h.load('small_dataset')
lstm_l.load('weak_supervision')

[LSTM] Loaded model <small_dataset>
[LSTM] Loaded model <weak_supervision>


In [5]:
session = SnorkelSession()

In [6]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

In [7]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

In [8]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

In [9]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [10]:
p, r, f1 = lstm_l.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.307, Recall: 0.706, F1 Score: 0.428


/home/cperreault/anaconda3/envs/snorkel-extraction/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [11]:
p, r, f1 = lstm_h.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.157, Recall: 0.165, F1 Score: 0.161


In [45]:
kernel =  RBF(1, length_scale_bounds=[.1,1]) + WhiteKernel(noise_level=.1) #first attempt at defining a kernel. Not exactly sure what will be best for this or how to tell
gpc_high = GaussianProcessClassifier(kernel = kernel)

In [13]:
dev_features = lstm_h.feature_outputs(dev_cands,100)

In [14]:
dev_features = dev_features.reshape(2811,50)

In [15]:
dev_y = L_gold_dev.toarray()
dev_y[dev_y != 1] = 0

In [46]:
gpc_high.fit(dev_features.detach().numpy(),dev_y.reshape(2811,))

GaussianProcessClassifier(copy_X_train=True,
                          kernel=RBF(length_scale=1) + WhiteKernel(noise_level=0.1),
                          max_iter_predict=100, multi_class='one_vs_rest',
                          n_jobs=None, n_restarts_optimizer=0,
                          optimizer='fmin_l_bfgs_b', random_state=None,
                          warm_start=False)

In [17]:
test_features_h = lstm_h.feature_outputs(test_cands,100)

In [18]:
test_features_h.shape

torch.Size([135050])

In [19]:
135050/ 50

2701.0

In [47]:
uncertainty_h = []
pred_prob = gpc_high.predict_proba(test_features_h.detach().numpy().reshape(2701,50), uncertainty = uncertainty_h)
uncertainty_h = uncertainty_h[0]

In [48]:
preds = []
for x in pred_prob:
    if x[0] > .5:
        preds.append(1)
    else:
        preds.append(0)

In [49]:
uncertainty_h = np.array(uncertainty_h)

In [58]:
uncertainty_h.var()

0.049643794516626975

In [54]:
buckets = np.linspace(0,1.1, 14)

In [55]:
buckets

array([ 0.        ,  0.08461538,  0.16923077,  0.25384615,  0.33846154,
        0.42307692,  0.50769231,  0.59230769,  0.67692308,  0.76153846,
        0.84615385,  0.93076923,  1.01538462,  1.1       ])

In [39]:
test_y = L_gold_test.toarray()
test_y[test_y != 1] = 0

In [30]:
sum(1 for x in test_y if x ==0)
# for now just pass on the 0 ? not sure what that means in the gold label

86

In [56]:
for i in range(13):
    count = 0
    tp,fp,tn,fn = 0,0,0,0
    for j,p in enumerate(test_y):
        if uncertainty_h[j] >= buckets[i] and uncertainty_h[j] < buckets[i+1]:
            count += 1
            if p == 0 and preds[j] == 0:
                tn += 1
            elif p == 0 and preds[j] == 1:
                fp += 1
            elif p == 1 and preds[j] == 0:
                fn += 1
            else: #p == 1 and preds[j] == 1:
                tp += 1
    try:
        p = tp / (tp + fp)
    except:
        p = 0
    try:
        r = tp / (tp + fn)
    except: 
        r = 0
    try:
        f1 = 2/(1/p + 1/r)
    except:
        f1 = 0
    print ("Bucket {}: \n Count: {}\n F1: {}\n{},{},{},{}".format(buckets[i+1], count, f1,tp,fp,tn,fn))

Bucket 0.08461538461538462: 
 Count: 0
 F1: 0
0,0,0,0
Bucket 0.16923076923076924: 
 Count: 0
 F1: 0
0,0,0,0
Bucket 0.25384615384615383: 
 Count: 0
 F1: 0
0,0,0,0
Bucket 0.3384615384615385: 
 Count: 112
 F1: 0.11764705882352941
7,105,0,0
Bucket 0.42307692307692313: 
 Count: 191
 F1: 0.08040201005025126
8,183,0,0
Bucket 0.5076923076923077: 
 Count: 161
 F1: 0.0718562874251497
6,155,0,0
Bucket 0.5923076923076923: 
 Count: 192
 F1: 0.11764705882352941
12,180,0,0
Bucket 0.676923076923077: 
 Count: 182
 F1: 0.12903225806451613
12,162,8,0
Bucket 0.7615384615384616: 
 Count: 164
 F1: 0.178343949044586
14,127,21,2
Bucket 0.8461538461538463: 
 Count: 217
 F1: 0.13145539906103285
14,182,18,3
Bucket 0.9307692307692308: 
 Count: 416
 F1: 0.15267175572519084
30,326,53,7
Bucket 1.0153846153846153: 
 Count: 1066
 F1: 0.1592920353982301
81,833,130,22
Bucket 1.1: 
 Count: 0
 F1: 0
0,0,0,0


In [52]:
gpc_high.score(dev_features.detach().numpy(),dev_y)

0.97082888651725363

In [53]:
gpc_high.kernel_

RBF(length_scale=1) + WhiteKernel(noise_level=1e-05)